# 이미지 포맷 및 사이즈 일괄 변경 프로그램 만들기

* 난이도 : ★★★★☆☆☆☆☆☆
* 필요라이브러리: pillow, os, argparse


* 이전강좌인 이미지 포맷 변경프로그램을 확장해서 사이즈까지 변경가능하게 기능을 추가합니다.
* 이미지 처리 기능을 클래스로 만들어 모듈화 시켜 사용하도록 하겠습니다.

## 클래스 화

* 클래스내 필요한 기능
    - 대상 폴더를 탐색할 재귀함수 (어떤 폴더를 탐색해야 할지)
    - 이미지 리사이즈 기능 (어떤 크기로 리사이즈를 해야할지)
    - 이미지 포맷을 바꿔서 저장할 기능 (어떤 포맷으로 저장해야할지)
    
* 괄호안의 내용은 클래스 생성시 아규먼트로 받도록 합니다.

In [ ]:
'''
파일명: myclass.py 
'''

from PIL import Image
import os

class MyImage():
    def __init__(self, **kwargs):
        '''클래스 생성자 함수'''
        
        # folder : 대상 폴더
        self.forder = kwargs.get("folder", None)
        # resize : 리사이즈를 할지 말지 여부
        self.resize = kwargs.get("resize", False)
        
        # resize 할때 축소값(resize_width, resize_height)
        self.r_width = kwargs.get("r_width", 500)
        self.r_height = kwargs.get("r_height", 500)
        
        # 포맷 변경시 대상 확장자
        self.ext = kwargs.get("ext", None)
        
        # 변경된 이미지를 저장할 폴더
        # 각 이미지 하위에 이 이름의 폴더를 생성해서 그 안에 저장합니다.
        self.newfolder = "__convert__"
        
        # 운영체제에 따라 폴더를 구분자가 다르기 때문에
        # 윈도우는 \\ 리눅스는 / 라서...
        self.path_separator = "\\"
    
    def is_valid_image(self, filename):
        '''이미지가 올바른 포맷인지 확인하는 함수
        
        Args:
            filename (str) : 이미지경로
        
        Returns:
            bool : True or False
        '''
        try:
            img = Image.open(filename)
            img.verify()
            img.close()
            return True
        except:
            return False
    
    def change_format(self, img, filename, ext):
        '''filename 에 해당하는 파일을 ext 확장자로 새롭게 포맷을 변경하는 함수
        Args:
            img (Image) : 오픈된 상태의 Image 객체
            filename (str) : 원본 이미지 경로
            ext (str) : 파일 포맷 확장자 (.jpg, .png, .gif)
        '''
        # 해당 이미지 파일 하위에 convert 폴더에 저장을 합니다.
        new_folder = os.path.split(filename)[0] + self.path_separator + self.newfolder
         
        # convert 폴더가 존재 하지 않으면 새로 생성합니다.
        if not os.path.exists(new_folder):
            os.mkdir(new_folder)

        src_filename = os.path.splitext(filename)[0]
        new_filename = new_folder + self.path_separator + src_filename.split(self.path_separator)[-1] + ext
        
        # 해당 이미지를 새로운 경로에 새로운 확장자로 저장합니다.
        img.save(new_filename)
        
    
    def resize_image(self, filename):
        '''인자로 넘어온 filename 의 파일을 리사이즈 합니다.
        리사이즈시 이미지 비율을 기준으로 세로 이미지는 세로를 기준으로 하고
        가로 이미지는 가로를 기준으로 리사이즈를 합니다.
        예를 들어 500 * 300 이미지를 리사이즈 하는데 기준값(r_width, r_height) 이 200 * 200이면
        가로 이미지기 때문에 500 / 200 = 2.5 니까 세로도 2.5 비율만큼만 축소하게 됩니다.
        
        Args:
            filename (str) : 이미지 원본 파일 경로
            
        Returns:
            pil_image : 리사이즈된 상태의 Image 객체를 리턴합니다.
        '''
        img = Image.open(filename)
        
        # 이미지의 width, height 을 구합니다.
        width, height = img.size
        
        # height 이 큰 경우는 세로 이미지
        if width < height:
            # 이미지 리사이즈 비율은 원본높이 / 대상높이
            # 예를 들어 원본 높이가 500인 이미지를 100으로 줄이게 되면 5배가 줄어든거니까
            # 이미지의 폭도 5배를 줄여야 이미지 비율 그대로 리사이즈가 되게 됩니다.
            aspect = height / self.r_height
            
            # 새로운 이미지의 폭은 원본 폭 사이즈 / 세로 리사이즈된 비율
            new_width = int(width / aspect)
            
            # 세로 이미지는 리사이즈 세로 높이값 그대로 
            new_height = self.r_height
        else:
            aspect = width / self.r_width
            new_width = self.r_width
            new_height = int(height / aspect)

        # 뉴사이즈는 new_width, new_height 의 튜플형태입니다.
        new_size = (new_width, new_height)
        
        # 새로운 사이즈 정보로 resize() 함수를 호출하고 바로 save 합니다.
        return img.resize(new_size)

    def search_dir(self, dirname):
        '''대상 폴더 내의 파일과 서브 폴더를 탐색하는 재귀 함수

        Args:
            dirname (str): 탐색 대상 폴더 경로

        Returns:
            list : 탐색한 결과 파일 리스트를 리턴
        '''
        # 결과를 리턴할 리스트 변수
        result_file_lists = []

        # dirname 의 경로의 파일과 폴더 목록을 구합니다.
        filenames = os.listdir(dirname)

        # 파일과 폴더 목록 반복문
        for filename in filenames:
            # filename 에는 os.listdir(dirname) 에서 dirname 이후의 경로값과 파일명만 존재하기 때문에
            # 전체 풀 경로를 얻기 위해선 dirname 과 filename 을 합쳐야 합니다.
            full_filepath = os.path.join(dirname, filename)

            # full_filepath 가 디렉토리라면..
            if os.path.isdir(full_filepath):
                # 폴더이름이 self.convertfoldername 인경우에는 탐색하지 않습니다.
                # 이미 변경된 이미지를 또 변경하는걸 막기 위해서 입니다.
                if filename == self.newfolder:
                    continue
                    
                # search_dir 인 자기 자신 함수를 다시 호출합니다.(재귀함수)
                # 여기서 중요한건 클래스 내부의 함수기 때문에
                # self.search_dir() 함수를 호출해야 한다는 것 입니다.
                result_file_lists.extend(self.search_dir(full_filepath))
            # 파일인 경우 result_file_lists 에 추가
            else:
                result_file_lists.append(full_filepath)

        # 결과 목록 리턴
        return result_file_lists
    
    def start(self):
        '''최종 기능을 동작시키는 함수'''
        
        # 리사이즈, 포맷변경된 카운트
        cnt_resize = 0
        cnt_format = 0
        
        # 포맷변경도 없고 리사이즈도 없으면 할일이 없습니다. 그냥 종료
        if self.ext is None and self.resize is False:
            return (cnt_resize, cnt_format)
            
        # 대상 폴더에서 모든 파일과 하위 폴더의 파일 목록을 구합니다.
        file_list = self.search_dir(self.forder)
        
        # 파일목록 요소 반복
        for file in file_list:
            
            # 이미지 포맷 유효성 체크 실패시 반복문을 반복합니다.
            if not self.is_valid_image(file):
                continue
                
            # 클래스 생성시 resize 옵션이 True 면
            if self.resize:
                # 이미지를 리사이즈 하여 Image 객체로 받습니다.
                img = self.resize_image(file)
                cnt_resize += 1
            else:
                # 리사이즈 옵션이 False 면 그냥 open 합니다.
                img = Image.open(file)
            
            # 확장자 변경은 없지만 리사이즈만 한 경우에느
            # ext 값을 현재 파일의 확장자로 그냥 저장합니다.
            if self.ext is None:
                # c:\a\b\c\d\e\123.jpg 인 값에서 123.jpg 만 빼오려고
                ext = str(file.split(self.path_separator)[-1]).split(".")[-1]
            else:
                ext = self.ext
                cnt_format += 1
                
            # 확장자가 . 으로 시작하지 않는경우 . 을 더해줘야함
            if ext[0] != ".":
                ext = "." + ext
            
            self.change_format(img, file, ext)
                
        return (cnt_resize, cnt_format)
                

## 메인 프로그램
* 메인프로그램에서는 위에서 생성한 클래스를 import 하여 사용합니다.
* 실행시 여러가지 실행인자값으로 클래스에 필요한 값을 설정합니다.
* 위에서 생성한 클래스 파일은 myclass.py 파일로 같은 경로에 존재 해야 합니다.

In [ ]:
# 위에서 생성한 클래스는 myclass.py 파일안에 있는 MyImage 라는 클래스 입니다.
from myclass import MyImage

# 실행시 인자값을 사용하기 위한 라이브러리
import argparse

if __name__ == "__main__":
    parse = argparse.ArgumentParser()
    parse.add_argument("-f", type=str, help="[대상폴더]")
    parse.add_argument("-e", type=str, help="[변경될 확장자]")
    parse.add_argument("-r", action="store_true", help="[옵션:리사이징]")
    parse.add_argument("-rw", type=int, default=500, help="[옵션:리사이징 width]")
    parse.add_argument("-rh", type=int, default=500, help="[옵션:리사이징 height]")
    args = parse.parse_args()

    # 대상폴더 f 값은 필수 값이고, 확장자변경(e) 나 리사이즈(r) 둘 중 하나는 설정되야 합니다.
    if args.f and (args.e or args.r):
        # 클래스를 생성합니다.
        myimg = MyImage(folder=args.f,      # 대상폴더 c:\temp
                        ext=args.e,         # 포맷변경 .gif
                        resize=args.r,      # 리사이즈 True or False
                        r_width=args.rw,    # 리사이즈 width 300
                        r_height=args.rh)   # 리아시즈 height 300

        # MyImage의 start() 함수 호출
        # 결과값은 (리사이즈 카운트, 포맷변경 카운트) 튜플
        cnt_resize, cnt_ext = myimg.start()
        print("리사이즈: {}, 포맷변경: {}".format(cnt_resize, cnt_ext))
    else:
        print("사용법: python change.py -f c:\\temp -e .png -r <옵션:리사이즈> -rw 500 -rh 500")